<a href="https://colab.research.google.com/github/SaswatD27/21cs460_group01/blob/main/Code/P2PModelMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import time
import numpy as np
import os
import tensorflow as tf
#from tensorflow_federated import federated_mean
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from copy import deepcopy
from statistics import mean


#print(x_cord, y_cord)

In [ ]:
#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
#emnist_train, emnist_test = datasets.emnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
img_rows, img_cols = 28, 28

In [ ]:
K=100
n_k_dev= np.random.randint(100,600,K)
#n_k=[600 for i in range(K)]
n_tot=sum(n_k_dev)
client_images=deepcopy(list(range(len(train_images))))
random.shuffle(client_images)
#print(lenMNIST)
x_train=[]
y_train=[]
'''
for i in range(K):
  x_train.append([])
  y_train.append([])
  for j in range(n_k[i]):
    k=client_images.pop()
    x_train[i].append(train_images[k])
    y_train[i].append(train_labels[k])
'''
indices_dev=[]
for i in range(K):
  indices_dev.append([])
  for j in range(n_k_dev[i]):
    k=client_images.pop()
    indices_dev[i].append(k)  

In [ ]:
'''
def SGD_(m, n, iteration, x_cord, y_cord, alpha):  # Performs client side SGD
    for p in range(iteration):
        for i in range(n):
            j = random.randint(0, n - 1)
            x_i = x_cord[j]
            y_a = y_cord[j]
            y_p = np.dot(m, x_i)
            dm = 2 * x_i * (y_p - y_a)
            m = m - alpha * dm
    return m
'''

def no_of_pts(id):
    return n_k_dev[id]

In [ ]:
#works!
def create_DNN():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=32, activation='relu'))
  #model.add(tf.keras.layers.Dropout(rate=0.2))
  #model.add(tf.keras.layers.BatchNormalization())
  #model.add(tf.keras.layers.Dense(units=512, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=10, activation='softmax')) 
  #model.add(tf.keras.layers.BatchNormalization())
  return model

In [ ]:
server_model=create_DNN()

In [ ]:
def device_local_learning(ini_model,id): #Conducts local training for SGD and outputs weight vector
    model=ini_model
    model.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    history = model.fit(train_images[indices_dev[id]],train_labels[indices_dev[id]], epochs=20, validation_data=(test_images, test_labels), verbose=0)
    #print(id," done.")
    save_weight(model, n_k_dev[id], id)

In [ ]:
def save_weight(w,n_k,id):
    n_k_devices[id]=n_k
    fl = str(id)
    #file = open(fl + "_wn.dat", "w")
    w.save_weights(fl + "_wn")
    #file.close()

In [ ]:
def avg_weights(client_frac,client_models,n_clients_round):
  client_models=np.array(client_models)
  res=client_frac[0]*np.array(client_models[0].get_weights())
  #print(res)
  for i in range(1,n_clients_round):
    res+=client_frac[i]*np.array(client_models[i].get_weights())
  return res

In [ ]:
#FED_AVG_DNN : WORKS NOW
def fed_avg_DNN(client_models, client_n):
  fin_model=create_DNN()
  fin_model.set_weights(0*np.array(fin_model.get_weights()))
  client_frac=np.zeros(len(client_n))
  for i in range(len(client_n)):
    client_frac[i]=(client_n[i])*(1/np.sum(client_n))
  #print("Sum of client_frac = ",np.sum(client_frac)," client_frac = ",client_frac)
  #client_weights_sum=np.sum(client_models)
  #for layer in server_model.layers:
  #  layer.set_weights([np.dot(client_frac,client_model.get_weights()[0]),np.dot(client_frac,client_model.get_weights()[1])])
  #server_model.set_weights([np.dot(client_frac,[m.get_weights()[0] for m in client_models]),np.dot(client_frac,[m.get_weights()[1] for m in client_models])])
  fin_model.set_weights(avg_weights(client_frac,client_models,len(client_frac)))
  return fin_model

In [ ]:
def request_dat(initiate):
    w_n=create_DNN()
    fl = str(initiate)
    w_n.load_weights(fl + "_wn").expect_partial()
    n_k_new= n_k_devices[initiate]
    return w_n,n_k_new

In [ ]:
def communicate(w_o, training_subset,n_k):
    comm_len= np.random.randint(1,max(len(training_subset),2))
    call_to = np.random.choice(training_subset,comm_len)
    print(call_to, "---", w_o)
    w_=[]
    n_=[]
    w_.append(w_o)
    n_.append(n_k)
    for i in call_to:
        w_w,n_w=request_dat(i)
        w_.append(w_w)
        n_.append(n_w)
    w_new=fed_avg_DNN(w_,n_)
    w_o.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])
    w_new.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])
    w_new.save_weights("w_new")
    err_o=1-w_o.evaluate(test_images,  test_labels, verbose=2)[1]
    err_1=1-w_new.evaluate(test_images,  test_labels, verbose=2)[1]
    if err_1<err_o:
        return w_new
    else:
        return w_o

In [ ]:
def p2p_learn(initiate,training_subset):
    w_n,n_k_new=request_dat(initiate)
    training_subset_c=training_subset.copy()
    training_subset_c=training_subset_c.tolist()
    training_subset_c.remove(initiate)
    w_new=communicate(w_n,training_subset_c,n_k_new)
    return w_new

In [ ]:
def P2P_new_fed1(K, C, server_model, n):
    training_subset = np.random.choice(all_device, int(C * K), replace=False)
    print(training_subset)
    ini_model=create_DNN()
    try:
        ini_model.load_weights("w_new")
    except:
        ini_model=server_model
    for j in training_subset:
        device_local_learning(ini_model,j)
    print("Initial Model sent.")
    w_new=[]
    acc=[]
    for i in training_subset:
        w_new.append(p2p_learn(i,training_subset))
        '''
        for i in training_subset:
            i = str(i)
            os.remove(i + "_wn")
        '''
    for w in w_new:
      acc.append(w.evaluate(test_images,  test_labels, verbose=0)[1])
    print("Accuracies for new models via P2P are",acc,"\n Max Accuracy = ",max(acc),"\n Min Accuracy = ",min(acc),"\n Mean Accuracy = ",np.mean(acc))
    ind_acc=[i for i in range(len(acc))]
    plt.scatter(ind_acc,acc)
    plt.xlabel('Clients in Training Sample')
    plt.ylabel('Accuracy')
    plt.title('P2P Model : Accuracy of Individual Clients')
    plt.show()
    ini_model.load_weights("w_new")
    print(ini_model.evaluate(test_images,  test_labels, verbose=0)[1])

In [ ]:
#To run another subsequent round, simply hit run again; prior weights, if any, are saved.
K = 100
C = 0.25
B= int(K * C)
#error_ = 10 ** (-4)
#w_i = np.random.rand(len(m))
n = 20
#alph_ = 0.01  # local learning rate

'''
device_list = []
for q in range(B):
    device_list.append('device_%i' % (q + 1))
'''
n_k_devices=np.zeros(K)
all_device=np.arange(K)


time1 = time.time()
P2P_new_fed1(K, C, server_model, n)
time2 = time.time()
print("Time taken for P2P federated learning", time2 - time1)
print(n_k_devices)


In [ ]:
#Plot
max_acc=[93.83999705314636,94.30999755859375,94.8199987411499,94.9999988079071,94.98000144958496,95.09000182151794,95.13999819755554]
min_acc=[92.66999959945679,92.76999831199646,93.97000074386597,93.80000233650208,94.37999725341797,94.24999952316284,93.80000233650208]
mean_acc=[93.49119901657105,94.02160000801086,94.4760000705719,94.62239980697632,94.7564001083374,94.24999952316284,94.90360021591187]
global_acc=[93.79000067710876,94.24999952316284,94.74999904632568,94.70999836921692,94.87000107765198,94.85240006446838,94.91999745368958]
x_cords_plt=range(1,8)
plt.plot(x_cords_plt,max_acc, label='Max Cluster Acc')
plt.plot(x_cords_plt,global_acc, label='Shared Global Model Acc')
plt.plot(x_cords_plt,mean_acc, label='Mean Cluster Acc')
plt.plot(x_cords_plt,min_acc, label='Min Cluster Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')